## Rendering virtual materials with Blender

Make sure the attached **materials_artist.blend** file is available. By default, images are saved in the **/renders** folder.

In [9]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import piexif
import piexif.helper
import json
import glob

import warnings
warnings.filterwarnings('ignore')

def read_metaImage(root):
    '''Reads metadata embedded in a JPG image.'''
    exif_dict = piexif.load(root)
    user_comment = piexif.helper.UserComment.load(exif_dict["Exif"][piexif.ExifIFD.UserComment])
    d = json.loads(user_comment)
    return d

def render_image(file, render_params, material_params, texture_params, suzanne=False):
    '''Generates a texture image based on selected parameters (render properties).'''
    # Set up temporary buffer directory
    base_root = os.getcwd()
    dst_root = root = os.path.abspath(os.path.join(base_root, f'renders/{file}.jpg'))
    root = os.path.abspath(os.path.join(base_root, f'renders/buffer/{file}.jpg'))
    width = render_params.get('width')
    height = render_params.get('height')
    
    # Save empty image in buffer
    Image.fromarray(np.zeros((width, height, 3), dtype=np.uint8)).save(root)
    
    all_params = {
        'render_params':render_params,
        'material_params':material_params,
        'texture_params':texture_params,
    }
    
    # Embed metadata (render properties)
    exif_dict = piexif.load(root)
    exif_dict["Exif"][piexif.ExifIFD.UserComment] = piexif.helper.UserComment.dump(json.dumps(all_params), encoding="unicode")
    piexif.insert(piexif.dump(exif_dict), root)
    
    # Run blender file to render image
    if suzanne:
        !blender -b suzanne_artist.blend --python-text suzanne_engine
    else:
        !blender -b materials_artist.blend --python-text texture_engine
    
    # Remove temporary file
    os.remove(root)
    
    # Return rendered image
    im = np.array(Image.open(os.path.splitext(dst_root)[-2]+'.jpg'))[...,:3]
    
    return im, dst_root

def render_batch(n, width=150, height=150, samples=32):
    textures = {'Brick':1, 'Checker':1, 'Magic':2, 'Noise':2, 'Wave':2}
    for k in range(n):
        tex = np.random.choice(list(textures.keys()))
        n_param = textures[tex]
        im, root = render_image(
            file=f'{k}', 
            render_params={
                'width':width,
                'height':height,
                'samples':samples,
            },
            material_params={
                'glossy_mix':0.0,
                'roughness':0.0,
                'R1':np.random.uniform(), 'G1':np.random.uniform(), 'B1':np.random.uniform(),
                'R2':np.random.uniform(), 'G2':np.random.uniform(), 'B2':np.random.uniform(),
            },
            texture_params={
                tex:list(np.random.uniform(size=n_param))
            },
        )

In [11]:
### Example: Render 10 images (/!\ ~2-5 sec. per image!)
render_batch(n=10, width=150, height=150, samples=32, )